This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'xxx' 
# i removed my actual API key

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import collections as col
import requests
import quandl
import json
import numpy as np
import statistics as stats
from bs4 import BeautifulSoup
# quandl.ApiConfig.api_key = "z4qJVzNVp5zuFNC47Dyz"  <-- quandl says we could do this instead

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2020-05-15&end_date=2020-05-15&api_key='+API_KEY
r = requests.get(url)
json_data = r.json()
for k in json_data.keys():
    print(k + ': ', json_data[k])
# data = quandl.get('FSE/AFX_X', start_date='2018-11-30', end_date='2018-11-30')  <-- and this
# print(data.head())

dataset:  {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-05-04) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-05-15T22:16:15.603Z', 'newest_available_date': '2020-05-15', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2020-05-15', 'end_date': '2020-05-15', 'data': [['2020-05-15', 82.25, 84.85, 80.65, 83.8, None, 219726.0, 18211281.9, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(type(json_data))
# Was I supposed to do anything else?

<class 'dict'>


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [11]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=z4qJVzNVp5zuFNC47Dyz'
r = requests.get(url)
json_data = r.json()

print("Columns of json_data['dataset']:")
print(json_data['dataset']['column_names'])
print()

open_min=99
open_max=0
for k in json_data['dataset']['data']:
    if type(k[1])==float:
        if k[1]>open_max:
            open_max=k[1]
#             print("New Max of Open in 2017: ",open_max)
        if k[1]<open_min:
            open_min=k[1]
#             print("New Min of Open in 2017: ",open_min)
print("Max Open in 2017: ",open_max)
print("Min Open in 2017: ",open_min)
    
difference_max=0
for k in json_data['dataset']['data']:
    if ((type(k[2])==float) & (type(k[3])==float)):
        if (k[2]-k[3])>difference_max:
            difference_max=(k[2]-k[3])
#         print("New Max Difference in a Single Day's High/Low: ",difference_max)
print("Max Difference in a Single Day's High/Low in 2017: ",difference_max)

# I'll assume they mean "consecutive" days for this
close_difference_max=0
close_yesterday=json_data['dataset']['data'][0][4]
for k in json_data['dataset']['data']:
    if type(k[4])==float:
        if (abs(close_yesterday-k[4])>close_difference_max):
            close_difference_max=(abs(close_yesterday-k[4]))
#             print("New Max Difference Between Two Consecutive Days' Close: ",close_difference_max)
        close_yesterday=k[4]
print("Max Difference Between Two Consecutive Days' Close in 2017: ",close_difference_max)

counter=0
total_traded_volume=0
for k in json_data['dataset']['data']:
    if type(k[6])==float:
        counter+=1
        total_traded_volume+=k[6]
mean_traded_volume=total_traded_volume/counter
print('Mean Traded Volume for 2017 :',mean_traded_volume)

median_traded_volume_list=[]
for k in json_data['dataset']['data']:
    if type(k[6])==float:
        median_traded_volume_list.append(k[6])
median_traded_volume = stats.median(median_traded_volume_list)
print('Median Traded Volume for 2017 :',median_traded_volume)

Columns of json_data['dataset']:
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']

Max Open in 2017:  53.11
Min Open in 2017:  34.0
Max Difference in a Single Day's High/Low in 2017:  2.8100000000000023
Max Difference Between Two Consecutive Days' Close in 2017:  2.559999999999995
Mean Traded Volume for 2017 : 89124.33725490196
Median Traded Volume for 2017 : 76286.0
